In [80]:
# relative import hack
import sys

sys.path.insert(0, "..")

from covid_dc.school_data import (
    scrape_articles_data,
    scrape_notifications_data,
    fix_multiple_dates,
    run_one_shot_fixes_html,
    parse_incidents,
    data_to_dataframe,
    fix_school_names,
    append_school_levels,
    # dl_and_read_pdfs,
    parse_pdfs,
    update_gsheet,
)

import pandas as pd

import seaborn as sns

%pylab inline

pd.options.display.max_rows = 250
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = None

Populating the interactive namespace from numpy and matplotlib


In [2]:
incidents = scrape_notifications_data()

<a href="https://dcpsreopenstrong.com/articles/dunbar-high-school-covid-19-notification-14/" rel="bookmark noopener noreferrer" target="_self">Dunbar High School: COVID-19 Notification</a>


In [3]:
incidents = fix_multiple_dates(incidents)

In [4]:
incidents = run_one_shot_fixes_html(incidents)

In [5]:
incidents = parse_incidents(incidents)

In [6]:
all_data = data_to_dataframe(incidents)

In [7]:
all_data = fix_school_names(all_data)

In [8]:
all_data = append_school_levels(all_data)

In [9]:
all_data.shape

(588, 7)

In [10]:
metadata = pd.read_csv("../data/input/DC_schools_metadata.csv")

In [11]:
all_data = all_data.merge(metadata, on="school", how="left")

In [12]:
all_data.shape

(588, 17)

In [13]:
all_data["ward"] = all_data["ward"].fillna("Unk")

In [14]:
all_data = all_data[all_data["letter_date"] >= "2021-10-10"]

In [15]:
archive_1 = pd.read_csv(
    "../data/input/archived_data_2021_2022_school_year_1.csv",
    parse_dates=["incident_date","letter_date"]
)

In [16]:
all_data = pd.concat([archive_1, all_data])

In [17]:
# this is craptacular but I'm leaving it for now because I may move
# it into the function that appends the metadata
update_gsheet(
    all_data[
        [
            "incident_date",
            "letter_date",
            "school",
            "cases_count",
            "school_level",
            "ward",
            "incident_text",
        ]
    ]
)

In [18]:
# Used for copypasta to gsheets if the above fails
# This will break if used on something without a notion of a clipboard
all_data[all_data.incident_date >= "2021-08-20"][
    [
        "incident_date",
        "letter_date",
        "school",
        "cases_count",
        "school_level",
        "ward",
        "incident_text",
    ]
].to_clipboard(index=False)


In [19]:
# Dropping sample data in a folder in the repo
all_data[all_data.incident_date >= "2021-08-20"].to_csv(
    "../data/output/school_covid_data_2021_2022_school_year.csv", index=False
)

In [20]:
asdf? =

SyntaxError: invalid syntax (1999948698.py, line 1)

## Below here is workspace for debugging and analysis

In [115]:
import requests

from bs4 import BeautifulSoup

import time

import textract
from dateparser.search import search_dates

In [116]:
# need to fix output format but whatever
def get_pdfs_per_school(school: str, sleep_time: int = 0) -> list:
    """
    The URLs seem to be consistent within a school even when they fall off of
    the page, so this rolls through them until it doesn't find something at the
    next int value. Returns a list of tuples (school, page_url, pdf_url)
    """
    url_base: str = "https://dcpsreopenstrong.com/articles/"
    url_mid: str = "-covid-19-notification"
    headers: dict = {"User-Agent": "Mozilla/5.0"}
    urls: list = []
    i: int = 1
    page_found: bool = True
    while page_found:
        if i == 1:
            page_url: str = url_base + school + url_mid + "/"
        else:
            page_url: str = url_base + school + url_mid + "-" + str(i) + "/"
        page: object = requests.get(page_url, headers=headers)
        soup: object = BeautifulSoup(page.content, "html.parser")

        try:
            pdf_url: str = soup.select("p+ .wp-block-file .wp-block-file__button")[0][
                "href"
            ]
            r = requests.get(pdf_url, headers=headers)
            # sloppy but w/ev
            with open("../data/input/pdf/" + pdf_url.split("/")[-1], "wb") as f:
                f.write(r.content)
            urls.append([school, page_url, pdf_url])
            i += 1
        except:
            page_found = False

        time.sleep(sleep_time)

    return urls

In [46]:
SCHOOLS = [
    "aiton-elementary-school",
    "amidon-bowen-elementary-school",
    "anacostia-high-school",
    "ballou-high-school",
    "ballou-stay-high-school",
    "bancroft-elementary-school",
    "banneker-high-school",
    "bard-high-school",
    "barnard-elementary-school",
    "beers-elementary-school",
    "boone-elementary-school",
    "brent-elementary-school",
    "brightwood-elementary-school",
    "brookland-middle-school",
    "browne-education-campus",
    "bruce-monroe-elementary-school",
    "bunker-hill-elementary-school",
    "burroughs-elementary-school",
    "burrville-elementary-school",
    "c-w-harris-elementary-school",
    "c-w-harris-elementary-school",
    "capitol-hill-montessori-school",
    "cardozo-education-campus",
    "cleveland-elementary-school",
    "columbia-heights-education-campus",
    "coolidge-high-school",
    "deal-middle-school",
    "dorothy-i-height-elementary-school",
    "drew-elementary-school",
    "duke-ellington-high-school",
    "dunbar-high-school",
    "eastern-high-school",
    "eaton-elementary-school",
    "eliot-hine-middle-school",
    "emery-building",
    "excel-academy",
    "f-s-key-elementary-school",
    "garfield-elementary-school",
    "garrison-elementary-school",
    "h-d-woodson-high-school",
    "hardy-middle-school",
    "hart-middle-school",
    "hd-cooke-elementary-school",
    "hearst-elementary-school",
    "hendley-elementary-school",
    "houston-elementary-school",
    "hyde-addison-elementary-school",
    "ida-b-wells-middle-school",
    "j-o-wilson-elementary-school",
    "janney-elementary-school",
    "jefferson-middle-school",
    "johnson-middle-school",
    "kelly-miller-middle-school",
    "ketcham-elementary-school",
    "kimball-elementary-school",
    "king-elementary-school",
    "kramer-middle-school",
    "lafayette-elementary-school",
    "langdon-elementary-school",
    "langley-elementary-school",
    "lasalle-backus-education-campus",
    "leckie-education-campus",
    "ludlow-taylor-elementary-school",
    "luke-c-moore-high-school",
    "macfarland-middle-school",
    "malcolm-x-elementary-school",
    "mann-elementary-school",
    "marie-reed-elementary-school",
    "maury-elementary-school",
    "mckinley-middle-school",
    "mckinley-tech-high-school",
    "military-road-early-learning-center",
    "miner-elementary-school",
    "moten-elementary-school",
    "murch-elementary-school",
    "nalle-elementary-school",
    "noyes-elementary-school",
    "oyster-adams-bilingual-school",
    "oyster-adams-bilingual-school",
    "patterson-elementary-school",
    "payne-elementary-school",
    "peabody-elementary-school",
    "phelps-ace-high-school",
    "plummer-elementary-school",
    "powell-elementary-school",
    "randle-highlands-elementary-school",
    "raymond-elementary-school",
    "river-terrace-education-campus",
    "ron-brown-college-preparatory-high-school",
    "roosevelt-high-school",
    "roosevelt-stay-high-school",
    "ross-elementary-school",
    "savoy-elementary-school",
    "school-within-school-goding",
    "school-without-walls",
    "school-without-walls-francis-stevens",
    "seaton-elementary-school",
    "shepherd-elementary-school",
    "simon-elementary-school",
    "smothers-elementary-school",
    "sousa-middle-school",
    "stanton-elementary-school",
    "stevens-early-childhood-center",
    "stevens-early-learning-center",
    "stoddert-elementary-school",
    "stuart-hobson-middle-school",
    "takoma-elementary-school",
    "thomas-elementary-school",
    "thomson-elementary-school",
    "truesdell-elementary-school",
    "tubman-elementary-school",
    "turner-elementary-school",
    "tyler-elementary-school",
    "van-ness-elementary-school",
    "walker-jones-education-campus",
    "watkins-elementary-school",
    "west-elementary-school",
    "wheatley-education-campus-school",
    "whittier-elementary-school",
    "whittier-elementary-urgent-message-on-virtual-learning-for-all-students",
    "wilson-high-school",
]

In [54]:
SCHOOLS

['wilson-high-school',
 'whittier-elementary-urgent-message-on-virtual-learning-for-all-students',
 'whittier-elementary-school',
 'wheatley-education-campus-school',
 'west-elementary-school',
 'watkins-elementary-school',
 'walker-jones-education-campus',
 'van-ness-elementary-school',
 'tyler-elementary-school',
 'turner-elementary-school',
 'tubman-elementary-school',
 'truesdell-elementary-school',
 'thomson-elementary-school',
 'thomas-elementary-school',
 'takoma-elementary-school',
 'stuart-hobson-middle-school',
 'stoddert-elementary-school',
 'stevens-early-learning-center',
 'stevens-early-childhood-center',
 'stanton-elementary-school',
 'sousa-middle-school',
 'smothers-elementary-school',
 'simon-elementary-school',
 'shepherd-elementary-school',
 'seaton-elementary-school',
 'school-without-walls-francis-stevens',
 'school-without-walls',
 'school-within-school-goding',
 'savoy-elementary-school',
 'ross-elementary-school',
 'roosevelt-stay-high-school',
 'roosevelt-high

In [55]:
%%time
pdf_out = [get_pdfs_per_school(s) for s in SCHOOLS]

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [39]:
pdf_list = []
for i in pdf_out:
    for j in i:
        pdf_list.append(list(j))

In [41]:
output_df = pd.DataFrame(pdf_list, columns=["school", "page_url", "pdf_url"])

In [43]:
output_df

,school,page_url,pdf_url
0,aiton-elementary-school,https://dcpsreopenstrong.com/articles/aiton-elementary-school-covid-19-notification/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.19_AitonES_Community-Notice.pdf
1,aiton-elementary-school,https://dcpsreopenstrong.com/articles/aiton-elementary-school-covid-19-notification-2/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.23_AitonES_Community_Notice.pdf
2,aiton-elementary-school,https://dcpsreopenstrong.com/articles/aiton-elementary-school-covid-19-notification-3/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.29_AitonES_Community_Notice.pdf
3,aiton-elementary-school,https://dcpsreopenstrong.com/articles/aiton-elementary-school-covid-19-notification-4/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/10/2021.10.05_AitonES_Community_Notice.pdf
4,aiton-elementary-school,https://dcpsreopenstrong.com/articles/aiton-elementary-school-covid-19-notification-5/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/10/2021.10.14_AitonES_Community_Notice-1.pdf
...,...,...,...
1224,wilson-high-school,https://dcpsreopenstrong.com/articles/wilson-high-school-covid-19-notification-18/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/2021.12.13_WilsonHS_Community_Notice.pdf
1225,wilson-high-school,https://dcpsreopenstrong.com/articles/wilson-high-school-covid-19-notification-19/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/2021.12.14_Wilson_Community_Notice.pdf
1226,wilson-high-school,https://dcpsreopenstrong.com/articles/wilson-high-school-covid-19-notification-20/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/2021.12.15_WilsonHS_Community_Notice-1.pdf
1227,wilson-high-school,https://dcpsreopenstrong.com/articles/wilson-high-school-covid-19-notification-21/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/2021.12.16_WilsonHS_Community_Notice.pdf


In [44]:
output_df.to_csv("../data/input/pdfs_19dec2021.csv", index=False)

In [70]:
output_df[output_df.school=="school-without-walls-francis-stevens"]

,school,page_url,pdf_url
957,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.16_SWWFS_Community_Notice.pdf
958,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-2/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.16_SWWFS_Community_Notice.pdf
959,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-3/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.19_SWW@FS_Community_Notice.pdf
960,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-4/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/09/2021.09.28_SWWFrancis_Community_Notice.pdf
961,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-5/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/11/2021.11.04_SWWFS_Community_Notice.pdf
962,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-6/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/11/2021.11.14_SWWFS_Community_Notice.pdf
963,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-7/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/11/2021.11.17_SWWFS_Community_Notice.pdf
964,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-8/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/21.12.06_SWWFS_Community_Notice.pdf
965,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-9/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/21.12.09_SWWFS_Community_Notice.pdf
966,school-without-walls-francis-stevens,https://dcpsreopenstrong.com/articles/school-without-walls-francis-stevens-covid-19-notification-10/,https://45biv636w8lm1agg3ozqtqg1-wpengine.netdna-ssl.com/wp-content/uploads/2021/12/2021.12.13_SWWFrancis_Community.pdf


## More workspace for developing PDF parser

In [117]:
import glob
from dateparser.search import search_dates

In [65]:
incidents = [textract.process("../data/input/pdf/2021.12.02_MauryES_Community_Notice.pdf")]

In [66]:
incidents = [re.sub(
                "\s+", " ", incidents[0].decode("utf-8").replace("\n", " ")
            ).strip()]

In [75]:
#parse_pdfs(incidents)

In [74]:
all_pdfs = glob.glob("../data/input/pdf/*.pdf")

In [118]:
# WIP
def parse_pdfs(incidents: list) -> dict:
    """
    Parse PDFs posted on the reopen strong website.
    Contains too much copypasta from the original parsing function,
    will DRY once it all works
    """
    schools = []
    letter_dates = []
    cases_counts = []
    incident_dates = []
    no_case_counts = []
    incidents_out = []

    for incident in incidents:
        # bp = boilerplate
        bp_0 = "Dear "
        bp_1 = ", This letter is to inform you"
        bp_2 = "medically cleared"

        main_text = re.search(r"(.*)" + re.escape(bp_2), incident).group(1)

        school = [
            re.search(re.escape(bp_0) + r"(.*)" + re.escape(bp_1), main_text).group(1)
        ]

        # this is a horrible hack around a bug in date_search()
        # dates_and_counts = re.search(re.escape(bp_1) + r"(.*)" + re.escape(bp_2), main_text).group(1)

        # Hacks because dateparser doesn't have options to ignore "on", "to", etc...
        # and the letter text does ratchet stuff like
        # "September 13 (Oyster campus), 2021" that dateparser thinks is two dates
        dates = [
            i
            for i in search_dates(main_text)
            if i[0] not in ["on", "to", "2021", "2022", "2023"]
        ]

        letter_date = [dates[0][1]]

        if len(dates) == 2:
            incident_date = [dates[1][1]]
            incident_out = [incident]
        else:
            incident_date = [d[1] for d in dates[1:]]

        incident_count = len(incident_date)
        letter_date = letter_date * incident_count
        school = school * incident_count
        incident_out = [incident] * incident_count

        # this will work until it doesn't
        number_words = [
            "zero",
            "one",
            "two",
            "three",
            "four",
            "five",
            "six",
            "seven",
            "eight",
            "nine",
            "ten",
            "eleven",
            "twelve",
            "thirteen",
            "fourteen",
            "fifteen",
            "sixteen",
            "seventeen",
            "eighteen",
            "nineteen",
            "twenty",
        ]
        cases_count_decode = dict(zip(number_words, list(range(21))))

        # list of all the number word appearances
        cases_count = [
            cases_count_decode.get(w)
            for w in main_text.split()
            if cases_count_decode.get(w)
        ]

        if len(cases_count) == 0:
            cases_count = [1] * incident_count
            no_case_count = [True] * incident_count
        elif len(cases_count) < incident_count:
            # this handles instances that use "two cases ... respectively".
            # spread cases over dates using floor division
            cases_count = [cases_count[0] // incident_count] * incident_count
            no_case_count = [False] * incident_count
        else:
            no_case_count = [False] * incident_count

        # hack to make sure cases count is never zero, which can happen when
        # the text uses weird constructions that boil down to "a case"
        cases_count = [max(c, 1) for c in cases_count]

        outputs = [
            school,
            letter_date,
            cases_count,
            incident_date,
            no_case_count,
            incident_out,
        ]

        if len({len(i) for i in outputs}) == 1:
            schools.extend(school)
            letter_dates.extend(letter_date)
            cases_counts.extend(cases_count)
            incident_dates.extend(incident_date)
            no_case_counts.extend(no_case_count)
            incidents_out.extend(incident_out)
        else:
            print(dates_and_counts, outputs)

    return {
        "school": schools,
        "letter_date": letter_dates,
        "cases_count": cases_counts,
        "incident_date": incident_dates,
        "no_case_count": no_case_counts,
        "incident_text": incidents_out,
    }

In [127]:
parsed_pdfs = []
for pdf in all_pdfs[:20]:
    try:
        contents = textract.process(pdf)
        contents = re.sub("\s+", " ", contents.decode("utf-8").replace("\n", " ")).strip()
    except:
        print("failed before parser: " + pdf)
    try:
        contents = parse_pdfs([contents])
        parsed_pdfs.append(contents)
    except:
        print("failed after  parser: " + pdf)
        print(contents[:400])

failed after  parser: ../data/input/pdf/2021.12.16_BeersES_Community_Notice-1.pdf
December 16, 2021 Dear Beers Elementary School, This letter is to inform you that three individuals who were last present at Beers on December 10, one individual on December 13, four individuals on December 14, and two individuals on December 15 have since reported a positive test for coronavirus (COVID-19). The individuals are no longer on campus and will not return until medically cleared. As de
failed after  parser: ../data/input/pdf/2021.12.12_WilsonHS_Community_Notice-1.pdf
December 12, 2021 Dear Wilson High School, This letter is to inform you that two individuals who were last present at Wilson on December 8 have since reported a positive test for coronavirus (COVID-19). The individuals are no longer on campus and will not return until medically cleared. As determined by the DCPS Contact Tracing team, DC Health, and in accordance with the district’s COVID-19 health
failed after  parser: ../data/inp

In [128]:
parsed_pdfs

[{'school': ['Ballou High School'],
  'letter_date': [datetime.datetime(2021, 9, 17, 0, 0)],
  'cases_count': [1],
  'incident_date': [datetime.datetime(2021, 9, 10, 0, 0)],
  'no_case_count': [True],
  'incident_text': ['September 17, 2021 Dear Ballou High School, This letter is to inform you that an individual who was last present at Ballou on September 10 has since reported a positive test for coronavirus (COVID-19). The individual is no longer on campus and will not return until medically cleared. As determined by the DCPS Contact Tracing team, DC Health, and in accordance with the district’s COVID-19 health and safety guidance, unless you have received instructions directly, your student has not been identified as a close contact of the positive individual and does not need to be tested or self-quarantine. Persons for whom we know were in close contact with the positive individual were notified and provided guidance on next steps, including how any students impacted would receive 

In [114]:
contents = textract.process("../data/input/pdf/2021.09.17_BallouHS_Community_Notice.pdf")
contents = re.sub("\s+", " ", contents.decode("utf-8").replace("\n", " ")).strip()
contents = parse_pdfs([contents])
parsed_pdfs.append(contents)

NameError: name 'search_dates' is not defined